# Adoucir ou Endurcir ?

In [ ]:
%run ../src/game.py
%run ../src/ipd.py
%run ../src/strategies.py
%run ../src/tools.py
g.prettyPrint()   # prisoner's dilemma

## Des meta-classes de stratégies

Quand on regarde un ensemble de stratégies on constate qu'il possède des caractéristiques générales qui permettent de les ranger en grandes familles. Les quatre familles que nous considérons sont la classe des stratégies initialement coopératives IC (elles ne prennent jamais l’initiative de trahir); la classe complémentaire à IC qui est celle des stratégies "spontanément agressives", SA (toute stratégie est donc soit IC soit SA); la classe des stratégies initialement agressives IA (elles ne prennent jamais l’initiative de coopérer); la classe complémentaire à IA qui est celle des stratégies "spontanément coopératives" SC (toute stratégie est donc soit IA soit SC). On note que toute stratégie IA est SA et que toute strategie IC est SC.

In [ ]:
def getAgressivityClasses(bag):
    aggressivity = {'IC':[], 'SC':[],'IA':[],'SA':[]}
    for strat in bag : 
        m1 = Meeting(g, strat, Periodic('C'), 100)
        m1.run()
        m2 = Meeting(g, strat, Periodic('D'), 100)
        m2.run()
        if m1.s1_score == 300:
             aggressivity['IC'] += [strat]
        if m1.s1_score > 300:
             aggressivity['SA'] += [strat]
        if m2.s1_score == 100:
             aggressivity['IA'] += [strat]
        if m2.s1_score < 100:
             aggressivity['SC'] += [strat]
    return aggressivity
        
    
print(getAgressivityClasses(getClassicals()))
        
        

L'aggressivité des stratégies avant et après simplification

In [ ]:
def createTab(bag):
    agr = getAgressivityClasses(bag)
    simplified = simplifyWithTournament(bag, [Periodic('CDCCDDC'), Periodic('DDCDCDD')], 10)
    agrS = getAgressivityClasses(simplified)
    tab = pd.DataFrame(
            np.nan, ["IC","SC","IA", "SA"], ["Before simplify","After simplify"]
        )
    for key in agr :
        tab.at[key,"Before simplify" ] = len(agr[key])
    for key in agrS :
        tab.at[key, "After simplify"] = len(agrS[key])
    pd.options.display.float_format = '{:,.0f}'.format
    return tab

createTab(getMem(1,1))


Maintenant, regardons avec Mem(1,2)

In [ ]:
createTab(getMem(1,2))

Quel est le classement moyen des stratégies de chaque classe ?

In [ ]:
bag = getMem(1,1)
e = Ecological(g, bag, 100)
e.run()
ranking = e.historic.iloc[e.generation].rank(0, method="min", ascending=False)
score = e.historic.iloc[e.generation]
agr = getAgressivityClasses(bag)

# Generate the data with mean of ranks and scores
tab = pd.DataFrame(
            np.nan, ["IC","SC","IA", "SA"], ["Mean of ranks","Mean of scores"]
        )
for key in agr:
    ranks = []
    scores = []
    for strat in agr[key]:
        ranks += [ranking[strat.name]]
        scores += [score[strat.name]]
        
    tab.at[key, "Mean of ranks"] = np.mean(ranks)
    tab.at[key, "Mean of scores"] = np.mean(scores)
pd.options.display.float_format = '{:,.0f}'.format   
print(tab)

# Generate the graph with mean of scores 
ranksIC, ranksSC, ranksIA, ranksSA = [], [], [], []
for i in range(e.generation):
    rIC = 0
    rSC = 0
    rIA = 0
    rSA = 0
    for key in agr:
        for strat in agr[key]:
            if key == "IC":
                rIC += e.historic.iloc[i][strat.name]
            if key == "SC":
                rSC += e.historic.iloc[i][strat.name]
            if key == "IA":
                rIA += e.historic.iloc[i][strat.name]
            if key == "SA":
                rSA += e.historic.iloc[i][strat.name]
    ranksIC += [np.sum(rIC)/len(agr['IC'])]
    ranksSC += [np.sum(rSC)/len(agr['SC'])]
    ranksIA += [np.sum(rIA)/len(agr['IA'])]
    ranksSA += [np.sum(rSA)/len(agr['SA'])]
                
        

plt.plot(ranksIC, label='IC')
plt.plot(ranksSC, label= 'SC')
plt.plot(ranksSA, label='SA')
plt.plot(ranksIA, label='IA')
plt.ylabel('Scores')
plt.xlabel('Generation')
plt.legend()
plt.show()
        